In [1]:
# 랜덤 시드 설정
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed

np.random.seed(0)
tf.random.set_seed(0)

n_batch = 64                            # 배치 크기
epochs = 100                            # 학습 epochs
latent_dim = 256                        # 단어 인코딩 축소 차원 
n_max_sample = 10000                    # 학습 시킬 최대 샘플 수
data_path = './data/eng-fra/fra.txt'    # 데이터 텍스트 파일 경로

# 데이터 불러오기
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
# 인풋, 타겟 데이터 정리
x_txts = []
y_txts = []
x_chars_uni = set()
y_chars_uni = set()
n_sample = min(n_max_sample, len(lines) - 1)    

for line in lines[:n_sample]:
    x_txt, y_txt, _ = line.split('\t')
    y_txt = '\t' + y_txt + '\n'
    x_txts.append(x_txt)
    y_txts.append(y_txt)
    
    for char in x_txt:
        if char not in x_chars_uni:
            x_chars_uni.add(char)
    for char in y_txt:
        if char not in y_chars_uni:
            y_chars_uni.add(char)

# 토큰 정리
x_chars_uni = sorted(list(x_chars_uni))
y_chars_uni = sorted(list(y_chars_uni))
n_encoder_tokens = len(x_chars_uni)
n_decoder_tokens = len(y_chars_uni)
print("유니크 인코더 토큰 글자 수: ", n_encoder_tokens)
print("유니크 디코더 토큰 글자 수: ", n_decoder_tokens)

max_encoder_seq_len = 0
for txt in x_txts:
    txt_len = len(txt)
    max_encoder_seq_len = max(txt_len, 
                              max_encoder_seq_len)
print("인코더 문장내 최대 문자 수: ", max_encoder_seq_len)

max_decoder_seq_len = 0
for txt in y_txts:
    txt_len = len(txt)
    max_decoder_seq_len = max(txt_len, 
                              max_decoder_seq_len)
print("디코더 문장내 최대 문자 수: ", max_decoder_seq_len)

# 토큰 인덱스
x_token_idx = {}
for idx, char in enumerate(x_chars_uni):
    x_token_idx[char] = idx
    
y_token_idx ={}
for idx, char in enumerate(y_chars_uni):
    y_token_idx[char] = idx
    
# 영행렬 만들기
encoder_x_data = np.zeros(
                            (len(x_txts), 
                             max_encoder_seq_len, 
                             n_encoder_tokens),
                        dtype='float32')
decoder_x_data = np.zeros(
                            (len(x_txts), 
                             max_decoder_seq_len, 
                             n_decoder_tokens),
                        dtype='float32')
decoder_y_data = np.zeros(
                            (len(x_txts), 
                             max_decoder_seq_len, 
                             n_decoder_tokens),
                        dtype='float32')
# 인풋 데이터 행렬
for i, x_txt in enumerate(x_txts):
    
    for t, char in enumerate(x_txt):
        encoder_x_data[i, t, x_token_idx[char]] = 1.
    encoder_x_data[i, t + 1:, x_token_idx[' ']] = 1.

# 타겟 데이터 행렬
for i, y_txt in enumerate(y_txts):
       
    for t, char in enumerate(y_txt):
        decoder_x_data[i, t, y_token_idx[char]] = 1.
        if t > 0:
            decoder_y_data[i, t - 1, y_token_idx[char]] = 1.
            
    decoder_x_data[i, t + 1:, y_token_idx[' ']] = 1.
    decoder_y_data[i, t:, y_token_idx[' ']] = 1.
    
# 인코더
encoder_inputs = Input(shape=(None, n_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# 디코더
decoder_inputs = Input(shape=(None, n_decoder_tokens))
decoder = LSTM(latent_dim, 
                return_sequences=True, 
                return_state=True)
decoder_outs, _, _ = decoder(decoder_inputs,
                             initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(n_decoder_tokens, 
                                      activation='softmax'))
decoder_outputs = decoder_dense(decoder_outs)

# 인코더 디코더
model = Model([encoder_inputs, decoder_inputs], 
              decoder_outputs)
model.summary()

# 모형 컴파일
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 학습
model.fit([encoder_x_data, decoder_x_data], decoder_y_data,
          batch_size=n_batch,
          epochs=epochs,
          validation_split=0.2)

# 추론 모형 생성
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, 
                         decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 리버스 인덱스
reverse_x_char_idx = {}
for char, idx in x_token_idx.items():
    reverse_x_char_idx[idx] = char
    
reverse_y_char_idx ={}
for char, idx in y_token_idx.items():
    reverse_y_char_idx[idx] = char

# 결과값 디코딩
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    y_seq = np.zeros((1, 1, n_decoder_tokens))
    y_seq[0, 0, y_token_idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [y_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_y_char_idx[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        y_seq = np.zeros((1, 1, n_decoder_tokens))
        y_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

# 최종 결과 확인
for seq_idx in range(100):
    x_seq = encoder_x_data[seq_idx: seq_idx + 1]
    decoded_sentence = decode_sequence(x_seq)
    print('-')
    print('Input sentence:', x_txts[seq_idx])
    print('Decoded sentence:', decoded_sentence)

유니크 인코더 토큰 글자 수:  71
유니크 디코더 토큰 글자 수:  92
인코더 문장내 최대 문자 수:  15
디코더 문장내 최대 문자 수:  59
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 71)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 92)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        335872      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
          

125/125 [==============================] - 18s 140ms/step - loss: 0.1404 - accuracy: 0.9567 - val_loss: 0.4938 - val_accuracy: 0.8767
Epoch 43/100
125/125 [==============================] - 18s 142ms/step - loss: 0.1365 - accuracy: 0.9577 - val_loss: 0.4970 - val_accuracy: 0.8764
Epoch 44/100
125/125 [==============================] - 17s 140ms/step - loss: 0.1325 - accuracy: 0.9589 - val_loss: 0.5028 - val_accuracy: 0.8760
Epoch 45/100
125/125 [==============================] - 17s 140ms/step - loss: 0.1292 - accuracy: 0.9599 - val_loss: 0.5057 - val_accuracy: 0.8769
Epoch 46/100
125/125 [==============================] - 17s 140ms/step - loss: 0.1256 - accuracy: 0.9610 - val_loss: 0.5148 - val_accuracy: 0.8749
Epoch 47/100
125/125 [==============================] - 19s 156ms/step - loss: 0.1225 - accuracy: 0.9622 - val_loss: 0.5168 - val_accuracy: 0.8754
Epoch 48/100
125/125 [==============================] - 18s 142ms/step - loss: 0.1194 - accuracy: 0.9628 - val_loss: 0.5214 - val_a

Epoch 98/100
125/125 [==============================] - 18s 142ms/step - loss: 0.0450 - accuracy: 0.9844 - val_loss: 0.7224 - val_accuracy: 0.8714
Epoch 99/100
125/125 [==============================] - 18s 142ms/step - loss: 0.0444 - accuracy: 0.9845 - val_loss: 0.7206 - val_accuracy: 0.8718
Epoch 100/100
125/125 [==============================] - 18s 143ms/step - loss: 0.0435 - accuracy: 0.9848 - val_loss: 0.7315 - val_accuracy: 0.8722
-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: Sale !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sen